# Apendix

Here we compute the transition matrix elements of a $d^4$ system like the one explored in chapter 5 for the ruthenate Ca$_2$RuO$_4$ (CRO). We compute the possible transitions of the type $d^4d^4\rightarrow d^3d^5$, applying some knowledge regarding the allowed ground and excited states of the CRO system. The transition matrix elements obtained are then used to compute the amplitude of certain transitions corresponding to the low energy optical excitations found in CRO.

With this in mind, we first explore a small example to test the logic of the computation. Then, we proceed to calculate the transition matrix elements of interest.

## Transitions for a two level and two site system with an electron on each site

For a two sites $(i,j)$ and two levels $\alpha, \beta$ system with one electron per site, the initial states are $d_i^1d_j^1$ configurations where one electron sits on the $\alpha$ level on each site which are $4$ in total. The final states are $d_i^0d_j^2$ configurations amounting to $5$ in total.

The representation of the states used is as follows: each level/orbital is composed of two elements of an array, one for each spin (up, down). This is to say that each element of the array corresponds to a creation operator $c^\dagger_{\alpha\sigma}$ when viewed from a second quantization notation. Thus, the first two elements of the array correspond to the $+$ and $-$ spin $\alpha$ level and the following two to the $+$ and $-$ spin $\beta$ level. To build the representation of the states of the combined two sites, we concatenate two single site arrays. This leads to an array where the first 4 elements correspond to the $i$ site and the second 4 ones correspond to the $j$ one.

For example, the state with spin up on both sites at the $\alpha$ level, equivalent to $c^\dagger_{i\alpha\uparrow}c^\dagger_{j\alpha\uparrow}$, is `[1 0 0 0 1 0 0 0]`. As a second example, we take the configuration where there is one spin up in each level at site $j$ whereas the site $i$ is empty `[0 0 0 0 1 0 1 0]`. For the remainder of this introduction, we will label those two configurations as `initial` and `final`, respectively.

In [1]:
initial = [1, 0, 0, 0, 1, 0, 0, 0]
final = [0, 0, 0, 0, 1, 0, 1, 0]

To compute if an optical transition between two states is possible or not, we first get some libraries to make this easier.

In [2]:
# Importing necessary extensions
import numpy as np
import itertools
import functools
import operator

# The use of type annotations requires Python 3.6 or newer
from typing import List

The question is, whether there is a transition matrix element between the aforementioned `initial` and `final` states. We can easily anser that with a yes, since the receiving level $\beta,+$ is empty in the initial state and no spin flip is involved when moving the particle from $\alpha,+$ to $\beta,+$. Thus, the question now is how to compute it in a systematic way. 

We can start by taking the `XOR` (exclusive or) operation between the constructed representations of the states. This means that we check where the changes between the states in question are located between the two states in question. Then, we check the positions (index) where we get a `1`, and if we find that both are odd or both are even, we can say that the transition is allowed. Whereas, if one is in odd and the other in even positions it is not allowed as it would imply a spin flip in the transition. This is equivalent as to write $<f|c^\dagger_{i\alpha'\sigma}c^\dagger_{j\beta'\sigma}|i>$.

Now we can go step by step codifying this procedure. First checking for the `XOR` operation and then asking where the two states differ.

In [3]:
# looking for the positions/levels with different occupation
changes = np.logical_xor(initial, final)
# obtaining the indexes of those positions
np.nonzero(changes)[0].tolist()  

[0, 6]

We can see that we get a change in positions $0$ and $6$ which correspond to $\alpha,+$ and $\beta,+$ in site $i$ and $j$, respectively. Now we apply modulo 2, which will allow us to check if the changes are in even or odd positions mapping even positions to $0$ whereas odd positions to $1$. Thus, if both are even or odd there will be just one unique element in the list otherwise there will be two unique elements.

In [4]:
modulo = 2
np.unique(np.remainder(np.nonzero(changes), modulo)).size == 1

True

Thus, in this case of chosen initial and finals states, the transition is allowed since both are even. We can wraps all of this logic in a function.

In [5]:
def is_allowed(initial: List[int], final: List[int]) -> bool:
    """
    Given an initial and final states as represented in a binary
    list, returns if it is allowed considering spin conservation.
    """
    return np.unique(
           np.remainder(
           np.nonzero(
           np.logical_xor(initial,final)), 2)).size == 1

Now we have a function that tells us if between two states an optical transition is possible or not. To recapitulate, we can recompute our previous case and then with a different final state that is not allowed since it involves a spin flip, e.g., `[0 0 0 0 0 1 1 0]`.

In [6]:
is_allowed(initial, final)

True

In [7]:
is_allowed(initial, [0, 0, 0, 0, 0, 1, 1, 0])

False

With this preamble, we are equiped to handle more complex cases. Given the chosen computational representation for the states, the normalization coefficients of the states are left out. Thus, one has to take care to keep track of them when constructing properly the transition matrix element in question later on.

## Ca$_2$RuO$_4$

Let us first explore the $d^4$ system. In a low spin $d^4$ system, we have only the t$_{2g}$ orbitals ($xy$, $yz$, $xz$) active which leads to a 6 elements representation for a site. Two neighboring states involved in a transition are concatenateed into a single array consisting of 12 elements.

For this, we create the function to generate the list representation of states given an amount of electrons and levels.

In [8]:
def generate_states(electrons: int, levels: int) -> List[List[int]]:
    """
    Generates the list representation of a given number of electrons 
    and levels (degeneracy not considered).
    """
    # create an array of length equal to the amount of levels 
    # with an amount of 1's equal to the number of electrons 
    # specified which will be used as a seed/template
    seed = [1 if position < electrons else 0 
            for position in range(levels)]
    # taking the seed state, we generate all possible permutations 
    # and remove duplicates using a set operation
    generated_states = list(set(itertools.permutations(seed)))
    generated_states.sort(reverse=True)
    return generated_states

With this we can generate states of 3, 4, and 5 electrons in a 3 level system with degeneracy 2 meaning 6 levels in total.

In [9]:
states_d3 = generate_states(3,6)
states_d4 = generate_states(4,6)
states_d5 = generate_states(5,6)

We can consider first the $d^4$ states and take a look at them.

In [10]:
states_d4

[(1, 1, 1, 1, 0, 0),
 (1, 1, 1, 0, 1, 0),
 (1, 1, 1, 0, 0, 1),
 (1, 1, 0, 1, 1, 0),
 (1, 1, 0, 1, 0, 1),
 (1, 1, 0, 0, 1, 1),
 (1, 0, 1, 1, 1, 0),
 (1, 0, 1, 1, 0, 1),
 (1, 0, 1, 0, 1, 1),
 (1, 0, 0, 1, 1, 1),
 (0, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 1),
 (0, 1, 1, 0, 1, 1),
 (0, 1, 0, 1, 1, 1),
 (0, 0, 1, 1, 1, 1)]

It is quite a list of generated states. But from this whole list, not all states are relevant for the problem at hand. This means that we can reduce the amount of states beforehand by applying the physical constrains we have. 

From all the $d^4$ states, we consider only those with a full $d_{xy}$ orbital and those which distribute the other two electrons in different orbitals as possible initial states for the Ca2RuO4 system. In our representation, this means only the states that have a 1 in the first two entries and no double occupancy in $zx$ or $yz$ orbitals.

In [11]:
possible_states_d4 = [
                      # select states that fulfill
                      list(state) for state in states_d4
                      # dxy orbital double occupancy
                      if state[0]==1 and state[1]==1
                      # dzx/dyz orbital single occupancy
                      and state[2] is not state[3]
                     ]
possible_states_d4

[[1, 1, 1, 0, 1, 0],
 [1, 1, 1, 0, 0, 1],
 [1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 1]]

We obtain 4 different $d^4$ states that fullfill the conditions previously indicated. From the previous list, the first and last elements correspond to states with $S_z=\pm1$ whereas the ones in the middle correspond to the two superimposed states for the $S=0$ state, namely, a magnon. These four states, could have been easily written down by hand, but the power of this approach is evident when generating and selecting the possible states of the $d^3$ configuration.

For the $d^3$ states, we want at least those which keep one electron in the $d_{xy}$ orbital since we know that the others states are not reachable with one movement as required by optical spectroscopy.

In [12]:
possible_states_d3 = [list(state) for state in states_d3 
                      if state[0]==1   # xy up occupied
                      or state[1]==1]  # xy down occupied
possible_states_d3

[[1, 1, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 0],
 [1, 1, 0, 0, 0, 1],
 [1, 0, 1, 1, 0, 0],
 [1, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 1],
 [1, 0, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 1, 1],
 [0, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1]]

In the case of the $d^5$ states, since our ground state has a doule occupied $d_{xy}$ orbital then it has to stay occupied.

In [13]:
possible_states_d5 = [list(state) for state in states_d5 
                      # xy up down occupied
                      if state[0]==1 and state[1]==1  
                     ]
possible_states_d5 

[[1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1]]

We could generate all $d^3d^5$ combinations and check how many of them there are.

In [14]:
def combine_states(first: List[List[int]], 
                   second: List[List[int]]) -> List[List[int]]:
    """
    Takes two lists of list representations of states and returns 
    the list representation of a two-site state.
    """
    # Producing all the possible final states. 
    # This has to be read from bottom to top.
    # 3) the single site representations are combined 
    # into one single two-site representation
    # 2) we iterate over all the combinations produced
    # 1) make the product of the given first and second 
    # states lists
    final_states = [
                    functools.reduce(operator.add, combination)  # 3)
                            for combination                      # 2)
                            in itertools.product(first, second)  # 1)
                   ]
    
    final_states.sort(reverse=True)
    
    return final_states

print("The number of combined states is: ", 
      len(combine_states(possible_states_d3,possible_states_d5)))

The number of combined states is:  64


We already saw in the previous section how we can check if a transition is allowed in our list codification of the states. Here we will make it a function slightly more complex to help us deal with generating final states.

In [15]:
def label(initial, final, levels, mapping):
    """Helper function to label the levels/orbitals involved."""
    changes = np.nonzero(np.logical_xor(initial, final))
    positions = np.remainder(changes, levels)//2
    return f"{mapping[positions[0][0]]} and {mapping[positions[0][1]]}"

def transition(initial: List[int], 
               final: List[List[int]], 
               debug = False) -> None:
    """
    This function takes the list representation of an initial double 
    site state and a list of final d3 states of intrest. 
    Then, it computes if the transition from the given initial state 
    to a compounded d3d5 final states are possible. 
    The d5 states are implicitly used in the function from those 
    already generated and filtered.
    """
    
    def process(final_states):
        # We iterate over all final states and test whether the 
        # transition from the given initial state is allowed
        for state in final_states:
            allowed = is_allowed(initial, state)
            if allowed:
                labeled = label(initial, 
                                state, 
                                6, 
                                {0: "xy", 1: "xz", 2: "yz"})
                print(f"     final state {state} allowed \
between {labeled}.")
            else:
                if debug:
                    print(f"     final state {state} not allowed.")
    
    d5 = list(possible_states_d5)
    print("From initial state {}".format(initial))
    print("d3d5")
    process(combine_states(final, d5))
    print("d5d3")
    process(combine_states(d5, final))    

With this, we can now explore the transitions between the different initial states and final states ($^4A_2$, $^2E$, and $^2T_1$ multiplets for the $d^3$ sector). Concerning the $d^4$ states, as explained in chapter 5, there is the possibility to be in the $S_z=\pm1$ or $S_z=0$. We will cover each one of them in the following.

What we are ultimately interested in is in the intensities of the transitions and thus we need the amplitudes since $I\sim\hat{A}^2$. We will go through each multiplet covering the ground states consisting of only $S_z=\pm1$ and then with the $S_z=0$.

## $^4A_2$

First, we will deal with the $^4A_2$ multiplet. The representations for the $|^4A_2,\pm3/2>$ states are given by

In [16]:
A2_32     = [[1,0,1,0,1,0]]  # 4A2 Sz=3/2
A2_neg_32 = [[0,1,0,1,0,1]]  # 4A2 Sz=-3/2

whereas the ones for the$|^4A_2,\pm1/2>$

In [17]:
A2_12     = [[0,1,1,0,1,0], [1,0,0,1,1,0], [1,0,1,0,0,1]]  # 4A2 Sz=1/2
A2_neg_12 = [[1,0,0,1,0,1], [0,1,1,0,0,1], [0,1,0,1,1,0]]  # 4A2 Sz=-1/2

Notice that the prefactors and signs are missing from this representation, and have to be taken into account when combining all the pieces into the end result.

### $S_z=\pm1$

Starting with the pure $S_z=\pm1$ as initial states, meaning $d_{\uparrow}^4d_{\uparrow}^4$ (FM) and $d_{\uparrow}^4d_{\downarrow}^4$ (AFM), we have the following representations:

In [18]:
FM       = [1,1,1,0,1,0,1,1,1,0,1,0]
AFM_up   = [1,1,1,0,1,0,1,1,0,1,0,1]
AFM_down = [1,1,0,1,0,1,1,1,1,0,1,0]

Handling the *ferromagnetic* ordering first, the allowed transitions from the initial state into the $|^4A_2,3/2>$ state are

In [19]:
transition(FM, A2_32)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0] allowed between yz and xy.


Comparing the initial and final states representations and considering the $|^4A_2,3/2>$ prefactor, we obtain that there are two possible transitions with matrix element $t_{xy,xz}$ and $t_{xy,yz}$. Each one is allowed twice from swapping the positions between $d^3$ and $d^5$.

Then, for the $|^4A_2,\pm1/2>$ states

In [20]:
transition(FM, A2_12)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


In [21]:
transition(FM, A2_neg_12)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


Thus, for the $|^4A_2,\pm1/2>$ states, there is no allowed transition starting from the FM initial ground state.

Repeating for both $^4A_2$ but starting from the *antiferromagnetic* state ($d^4_\uparrow d^4_\downarrow$) initial state we get

In [22]:
transition(AFM_up, A2_32)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3


In [23]:
transition(AFM_up, A2_12)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
     final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
     final state [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3


In [24]:
transition(AFM_up, A2_neg_12)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1] allowed between yz and xy.


We see that the AFM initial ground state has no transition matrix element for the $|^4A_2,3/2>$ state. Whereas transitions involving the $|^4A_2,\pm1/2>$ state are allowed. Once again, checking the prefactors for the multiplet and the initial ground state we get a transition matrix element of $t_{xy,xz}/\sqrt{3}$ and $t_{xy,yz}/\sqrt{3}$, twice each.

These are the same results as could have been obtained using simple physical arguments.

### $S_z=0$

The case of $S_z=0$ is handled similarly, the difference is that we get more terms to handle. We start with the $d_0^4d_\uparrow^4$ initial state and the $|^4A_2,\pm3/2>$ states. Since the $d_0^4$ is a superposition of two states, we will split it in the two parts.

Being $|f>$ any valid final state involving a combination (tensor product) of a $|d^3>$ and a $|d^5>$ states, and being $|i>$ of the type $|d^4_0>|d^4_\uparrow>$ where $|d^4_0>=|A>+|B>$, then the matrix element $<f|\hat{t}|i>$ can be split as $<f|\hat{t}|A>|d^4_\uparrow>+<f|\hat{t}|B>|d^4_\uparrow>)$.



In [25]:
S0_1 = [1, 1, 1, 0, 0, 1]  # |A>
S0_2 = [1, 1, 0, 1, 1, 0]  # |B>

d_zero_down = [1, 1, 0, 1, 0, 1]
d_zero_up   = [1, 1, 1, 0, 1, 0]

Thus, we append the $d^4_\uparrow$ representation to each part of the $d^4_0$ states. Then, checking for the transitions into the $|^4A_2,\pm3/2>$ $d^3$ state we get

In [26]:
transition(S0_1 + d_zero_up, A2_32)
transition(S0_2 + d_zero_up, A2_32)
print("\n\n")
transition(S0_1 + d_zero_up, A2_neg_32)
transition(S0_2 + d_zero_up, A2_neg_32)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0] allowed between yz and xy.



From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


Collecting the terms we get that for $|^4A_2, 3/2>$ there is no transitions into a $|d^3>|d^5>$ final state but there are transitions into two different $|d^5>|d^3>$ final states, one for each of the $|A>$ and $|B>$ parts. Thus, considering the numerical factors of the involved states, the amplitude in this case is $\frac{1}{\sqrt{2}}t_{xy,xz}$ and  $\frac{1}{\sqrt{2}}t_{xy,yz}$. In this case, the states involved in $|^4A_2, -3/2>$ do not show any allowed transition.

Now, we can perform the same procedure but considering the $d^4_\downarrow$ state.

In [27]:
transition(S0_1 + d_zero_down, A2_32)
transition(S0_2 + d_zero_down, A2_32)
print("\n\n")
transition(S0_1 + d_zero_down, A2_neg_32)
transition(S0_2 + d_zero_down, A2_neg_32)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3



From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1] allowed between xz and xy.


Here, we observe the same situation than before but swapping the roles between the $|^4A_2,\pm3/2>$ states. This means that the contribution of the $d^0 d^4_\uparrow$ is the same as the $d^0 d^4_\downarrow$ one.

Similarly, we can start from the $d^4_\uparrow d^0$ or the $d^4_\downarrow d^0$ which will also swap from transitions involving a $|d^5>|d^3>$ state to the $|d^3>|d^5>$ ones. The explicit computation is shown next for completeness.

In [28]:
transition(d_zero_up + S0_1, A2_32)
transition(d_zero_up + S0_2, A2_32)
print("\n\n")
transition(d_zero_up + S0_1, A2_neg_32)
transition(d_zero_up + S0_2, A2_neg_32)
print("\n\n")
transition(d_zero_down + S0_1, A2_32)
transition(d_zero_down + S0_2, A2_32)
print("\n\n")
transition(d_zero_down + S0_1, A2_neg_32)
transition(d_zero_down + S0_2, A2_neg_32)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3



From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3



From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
d5d3



From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
d5d3


Following the same procedure for the $|^4A_2, 1/2>$ states and $d^4_0d^4_\uparrow$ ground state

In [29]:
transition(S0_1 + d_zero_up, A2_12)
transition(S0_2 + d_zero_up, A2_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0] allowed between xz and xy.


Here we get some possible transitions to final states of interest. Here, we have to remember that the "receiving" $d3$ multiplet has three terms, which have to be added if present. For the $|d^3>|d^5>$ case there are two allowed transitions into $d^5$ states involving $t_{xy,xz}$ and $t_{xy,yz}$ for $|A>$ and $|B>$. From $|A>$ and $|B>$ we find computed terms that correspond to the same $d^5$ final state that have to be added.

Thus, considering the $1/\sqrt{2}$ and $1/\sqrt{3}$ prefactors for the states, each term has a factor of $1/\sqrt{6}$. Then, we obtain the total contributions $\sqrt{\frac{2}{3}}t_{xy,xz}$ and $\sqrt{\frac{2}{3}}t_{xy,yz}$ for transitions into $d^5_{xz/xy,\downarrow}$ in the $|d^3>|d^5>$ case, whereas for the $|d^5>|d^3>$ one, we obtain $\sqrt{\frac{1}{6}}t_{xy,xz}$ and $\sqrt{\frac{1}{6}}t_{xy,yz}$ for the final states involving $d^5_{xz,\uparrow}$ and $d^5_{xz,\uparrow}$ states, respectively.

And for the $|^4A_2, -1/2>$ state

In [30]:
transition(S0_1 + d_zero_up, A2_neg_12)
transition(S0_2 + d_zero_up, A2_neg_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


there is no transition found.

We repeat for $|d^4_\uparrow d^4_0>$ 

In [31]:
transition(d_zero_up + S0_1, A2_12)
transition(d_zero_up + S0_2, A2_12)
print("\n\n")
transition(d_zero_up + S0_1, A2_neg_12)
transition(d_zero_up + S0_2, A2_neg_12)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0] allowed between yz and xy.



From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3


Which is the same situation than before but swapping the position of the contributions as we already saw for the $|^4A_2, 3/2>$ case. For completeness we show the situation with $d^4_\downarrow$ as follows.

In [32]:
transition(S0_1 + d_zero_down, A2_12)
transition(S0_2 + d_zero_down, A2_12)
print("\n\n")
transition(d_zero_down + S0_1, A2_12)
transition(d_zero_down + S0_2, A2_12)
print("\n\n")
transition(S0_1 + d_zero_down, A2_neg_12)
transition(S0_2 + d_zero_down, A2_neg_12)
print("\n\n")
transition(d_zero_down + S0_1, A2_neg_12)
transition(d_zero_down + S0_2, A2_neg_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3



From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
d5d3



From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
     final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
     final state [0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
     final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
     final state [0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1] allowed between yz and xy.



From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 

Continuing with the $d^4_0d^4_0$ the situation gets more complicated since $<f|\hat{t}|d^4_0>|d^4_0>$ can be split as follows $<f|\hat{t}(|A>+|B>)(|A>+|B>)$ which gives 4 terms labeled $F$ to $I$. Thus, we construct the four combinations for the initial state and calculate each one of them to later sum them up.

In [33]:
F = S0_1 + S0_1
G = S0_1 + S0_2
H = S0_2 + S0_1
I = S0_2 + S0_2

First dealing with the $|^4A_2,\pm 3/2>$ states for the $d^3$ sector.

In [34]:
transition(F, A2_32)
transition(G, A2_32)
transition(H, A2_32)
transition(I, A2_32)

transition(F, A2_neg_32)
transition(G, A2_neg_32)
transition(H, A2_neg_32)
transition(I, A2_neg_32)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3


No transitions from the $d^4_0d^4_0$ state to $|^4A_2,\pm3/2>$.

And now repeating the same strategy for the $|^4A_2,1/2>$ state

In [35]:
transition(F, A2_12)
transition(G, A2_12)
transition(H, A2_12)
transition(I, A2_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0] allowed between yz and xy.


Here we have terms for both $|d^3>|d^5>$ and $|d^5>|d^3>$ and for each component of the initial state which can be grouped into which $d^5$ state they transition into. Terms pairs $F-H$ and $G-I$ belong together involving the $d^5_{xz\downarrow}$ and $d^5_{yz\downarrow}$ states, respectively.

Adding terms corresponding to $d^3$ multiplet participating and considering the prefactors, we get the terms $\frac{1}{\sqrt{3}}t_{xy,xz}$ and $\frac{1}{\sqrt{3}}t_{xy,yz}$.

And for completeness the $|^4A_2,-1/2>$ state

In [36]:
transition(F, A2_neg_12)
transition(G, A2_neg_12)
transition(H, A2_neg_12)
transition(I, A2_neg_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0] allowed between xz and xy.


For $|^4A_2,-1/2>$ states we obtain the same values than for $|^4A_2,1/2>$ but involving the other spin state.

Now we have all the amplitudes corresponding to transitions into the $^4A_2$ multiplet enabled by the initial states involving $S_z=0$, namely, $\uparrow 0+ 0\uparrow+ \downarrow 0 + 0\downarrow + 00$.

## $|^2E,a/b>$

First we encode the $|^2E,a>$ multiplet and check the $S_z=\pm1$ ground states

In [37]:
Ea = [[0,1,1,0,1,0], [1,0,0,1,1,0], [1,0,1,0,0,1]]
transition(AFM_down, Ea)
transition(AFM_up, Ea)
transition(FM, Ea)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0] allowed between xz and xy.
     final state [1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0] allowed between yz and xy.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
     final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
     final state [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


For the $|^2E,a>$ multiplet, only transitions from the AFM ground state are possible. Collecting the prefactors we get that the transition matrix element in $-\sqrt{2/3}t_{xy,xz}$ and $-\sqrt{2/3}t_{xy,yz}$ as could be easily checked by hand.

Then, for the $|^2E,b>$ multiplet 

In [38]:
Eb = [[1,0,1,0,0,1], [1,0,0,1,1,0]]
transition(AFM_down, Eb)
transition(AFM_up, Eb)
transition(FM, Eb)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


From the $S=\pm1$ initial states, no transitions possible to Eb.

We follow with the situation when considering the $S=0$. In this case, each initial state is decomposed in two parts resulting in 4 terms.

In [39]:
transition(S0_1 + S0_1, Ea)
transition(S0_1 + S0_2, Ea)
transition(S0_2 + S0_1, Ea)
transition(S0_2 + S0_2, Ea)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0] allowed between yz and xy.


Each one of the combinations is allowed, thus considering the prefactors of the $S_0$ and $|^2E,a>$ we obtain $\sqrt{\frac{2}{3}}t_{xy,xz}$ and $\sqrt{\frac{2}{3}}t_{xy,yz}$.

Doing the same for $|^2E,b>$

In [40]:
transition(S0_1 + S0_1, Eb)
transition(S0_1 + S0_2, Eb)
transition(S0_2 + S0_1, Eb)
transition(S0_2 + S0_2, Eb)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0] allowed between xz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xy and xz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0] allowed between yz and xy.


Adding all the contributions of the allowed terms we obtain, that due to the - sign in the $|^2E,b>$ multiplet, the contribution is 0.

We sill have to cover the ground state of the kind $d_0^4d_\uparrow^4$. As done previously, we again will split the $d_0^4$ in the two parts.

In [41]:
S0_1 = [1, 1, 1, 0, 0, 1]
S0_2 = [1, 1, 0, 1, 1, 0]

and then we add the $d^4_\uparrow$ representation to each one. Thus, for the $|^2E, Ea>$ $d^3$ multiplet we get

In [42]:
transition(S0_1 + d_zero_up, Ea)
transition(S0_2 + d_zero_up, Ea)
print("\n\n")
transition(d_zero_up + S0_1, Ea)
transition(d_zero_up + S0_2, Ea)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0] allowed between yz and xy.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0] allowed between xz and xy.



From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     fina

Here, both parts of the $S_z=0$ state contribute. Checking the prefactors for $S_z=0$ ($1/\sqrt{2}$) and $|^2E, Ea>$ ($1/\sqrt{6}$) we get a matrix element $\sqrt{\frac{2}{3}}t_{xy/xz}$.

Following for transitions into the $|^2E, Eb>$

In [43]:
transition(S0_1 + d_zero_up, Eb)
transition(S0_2 + d_zero_up, Eb)
print("\n\n")
transition(d_zero_up + S0_1, Eb)
transition(d_zero_up + S0_2, Eb)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xy and xz.
     final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xy and yz.
d5d3



From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1] allowed between yz and xy.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0] allowed between xz and xy.
     final state [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0] allowed between yz and xy.


## $|^2T_1,+/->$

This multiplet has 6 possible forms, $\textit{xy}$, $\textit{xz}$, and $\textit{yz}$ singly occupied

First we encode the $|^2T_1,+>$ multiplet with singly occupied $\textit{xy}$

In [44]:
T1_p_xy = [[1,0,1,1,0,0], [1,0,0,0,1,1]]
transition(AFM_down, T1_p_xy)
transition(AFM_up, T1_p_xy)
transition(FM, T1_p_xy)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


And for the $|^2T_1,->$

In [45]:
T1_n_xy = [[0,1,1,1,0,0], [0,1,0,0,1,1]]
transition(AFM_down, T1_n_xy)
transition(AFM_up, T1_n_xy)
transition(FM, T1_n_xy)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


In this case, there is no possible transition to states with a singly occupied $\textit{xy}$ orbital from the $\textit{xy}$ ordered ground state.

In [46]:
T1_p_xz = [[1,1,1,0,0,0], [0,0,1,0,1,1]]
transition(AFM_up, T1_p_xz)
transition(FM, T1_p_xz)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1] allowed between yz and xz.
     final state [1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1] allowed between yz and yz.
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


In [47]:
T1_p_yz = [[1,1,0,0,1,0], [0,0,1,1,1,0]]
transition(AFM_up, T1_p_yz)
transition(FM, T1_p_yz)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xz and xz.
     final state [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1] allowed between xz and yz.
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


We can see that the transitions from the ferromagnetic state are forbidden for the $xy$ orbitally ordered ground state for both $|^2T_1, xz\uparrow>$ and $|^2T_1, yz\uparrow>$ while allowing for transitions with amplitudes: $t_{yz,xz}/\sqrt{2}$, $t_{xz,xz}/\sqrt{2}$, $t_{xz,yz}/\sqrt{2}$, and $t_{yz,yz}/\sqrt{2}$.

For completeness, we show the transitions into the states $|^2T_1, xz\downarrow>$ and $|^2T_1, yz\downarrow>$ from the $\uparrow\uparrow$ and $\uparrow\downarrow$ ground states.

In [48]:
T1_n_xz = [[1,1,0,1,0,0], [0,0,0,1,1,1]]
transition(AFM_up, T1_n_xz)
transition(FM, T1_n_xz)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0] allowed between xz and yz.
     final state [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0] allowed between yz and yz.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


In [49]:
T1_n_yz = [[1,1,0,0,0,1], [0,0,1,1,0,1]]
transition(AFM_up, T1_n_yz)
transition(FM, T1_n_yz)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1] allowed between xz and xz.
     final state [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1] allowed between yz and xz.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3


## S=0

Now the challenge of addressing this multiplet when considering the $S=0$ component in the ground state.

In [50]:
S0_1 = [1, 1, 1, 0, 0, 1]
S0_2 = [1, 1, 0, 1, 1, 0]

T1_p_xz = [[1,1,1,0,0,0], [0,0,1,0,1,1]]
T1_p_yz = [[1,1,0,0,1,0], [0,0,1,1,1,0]]

First, we calculate for the $d^4_0d^4_\uparrow$ ground state. Again the $d^4_0$ state is split in two parts.

In [51]:
transition(S0_1 + d_zero_up, T1_p_xz)
transition(S0_2 + d_zero_up, T1_p_xz)
print("\n\n")
transition(S0_1 + d_zero_up, T1_p_yz)
transition(S0_2 + d_zero_up, T1_p_yz)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0] allowed between yz and xz.
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1] allowed between yz and yz.
d5d3
     final state [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0] allowed between yz and yz.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0] allowed between xz and yz.



From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0] allowed between yz and xz.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]
d3d5
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xz and xz.
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xz and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0] allowed between xz and xz.


And for $d^4_0d^4_\downarrow$ 

In [52]:
transition(S0_1 + d_zero_down, T1_p_xz)
transition(S0_2 + d_zero_down, T1_p_xz)
print("\n\n")
transition(S0_1 + d_zero_down, T1_p_yz)
transition(S0_2 + d_zero_down, T1_p_yz)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3



From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
d3d5
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
d3d5
d5d3


Thus, for final states with singly occupied $\textit{xz}$ multiplet, we obtain transitions involving $t_{yz,xz}/2$, $t_{yz,yz}/2$, $t_{xz,xz}/2$ and $t_{xz,yz}/2$ when accounting for the prefactors of the states.

For completeness, repeating for the cases $d^4_\uparrow d^4_0$ and $d^4_\downarrow d^4_0$

In [53]:
transition(d_zero_up + S0_1, T1_p_xz)
transition(d_zero_up + S0_2, T1_p_xz)
print("\n\n")
transition(d_zero_up + S0_1, T1_p_yz)
transition(d_zero_up + S0_2, T1_p_yz)
print("\n\n")
print("\n\n")
transition(d_zero_down + S0_1, T1_p_xz)
transition(d_zero_down + S0_2, T1_p_xz)
print("\n\n")
transition(d_zero_down + S0_1, T1_p_yz)
transition(d_zero_down + S0_2, T1_p_yz)

From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1] allowed between yz and yz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0] allowed between xz and yz.
     final state [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0] allowed between yz and yz.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0] allowed between yz and xz.
d5d3



From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1] allowed between xz and yz.
d5d3
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0] allowed between xz and xz.
d5d3
     final state [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0] allowed between xz and xz.
     final state [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0] allowed between yz and xz.






From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial 

In this case, considering the prefactors of the states involved, we obtain contributions $t_{yz,xy}/{2}$ and $t_{yz,yz}/{2}$, $t_{xz,xz}/{2}$, and $t_{xz,yz}/{2}$.

And at last $d^4_0d^4_0$

In [54]:
transition(S0_1 + S0_1, T1_p_xz)
transition(S0_1 + S0_2, T1_p_xz)
transition(S0_2 + S0_1, T1_p_xz)
transition(S0_2 + S0_2, T1_p_xz)
print("------------------------")
transition(S0_1 + S0_1, T1_p_yz)
transition(S0_1 + S0_2, T1_p_yz)
transition(S0_2 + S0_1, T1_p_yz)
transition(S0_2 + S0_2, T1_p_yz)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1] allowed between yz and xz.
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0] allowed between xz and yz.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
     final state [1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1] allowed between yz and yz.
d5d3
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
     final state [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0] allowed between yz and yz.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
------------------------
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
d3d5
d5d3
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
d3d5
d5d3
     final state [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0] allowed between xz and xz.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
d3d5
     final state [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1] allowed between xz and xz.
d5d3
From initial state [1, 1, 0

With contributions $t_{yz,xy}/{2\sqrt{2}}$ and $t_{yz,yz}/{2\sqrt{2}}$, $t_{xz,xz}/{2\sqrt{2}}$, and $t_{xz,yz}/{2\sqrt{2}}$.